<a href="https://colab.research.google.com/github/Sdapaul/Auto-Invest-Coin/blob/main/Analytics%20of%20Protein.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# @title 1. 필요한 라이브러리 설치
# 이 셀을 실행하면 분석에 필요한 도구들이 설치됩니다.
!pip install biopython py3Dmol requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 32.7 MB/s eta 0:00:00


In [9]:
# @title 2. 분석 및 시각화 도구 정의 (업그레이드 버전)
import os
import requests
import Bio.PDB
import py3Dmol
import numpy as np
from Bio.PDB import PDBList

# --- 1. 데이터 다운로드 클래스 (API 연동) ---
class ProteinFetcher:
    def __init__(self, download_dir="/content/protein_structures"):
        self.download_dir = download_dir
        if not os.path.exists(self.download_dir):
            os.makedirs(self.download_dir)

    def fetch_from_pdb(self, pdb_id):
        """RCSB PDB에서 실험 구조 다운로드"""
        pdb_id = pdb_id.upper()
        print(f"📥 [PDB] Fetching {pdb_id}...")
        pdbl = PDBList(verbose=False)
        try:
            file_path = pdbl.retrieve_pdb_file(pdb_id, pdir=self.download_dir, file_format='pdb')
            # 파일명 정리 (.ent -> .pdb)
            if os.path.exists(file_path):
                new_path = os.path.join(self.download_dir, f"{pdb_id}.pdb")
                if file_path != new_path:
                    os.rename(file_path, new_path)
                return new_path
        except Exception as e:
            print(f"❌ [PDB Error] {e}")
        return None

    def fetch_from_alphafold(self, uniprot_id):
        """AlphaFold DB API를 통해 최신 구조 URL을 찾아 다운로드"""
        uniprot_id = uniprot_id.upper()
        print(f"📥 [AlphaFold] Searching API for {uniprot_id}...")

        # 1. API에 메타데이터 요청
        api_url = f"https://alphafold.ebi.ac.uk/api/prediction/{uniprot_id}"
        try:
            r = requests.get(api_url)
            if r.status_code != 200:
                print(f"❌ [AlphaFold Error] ID {uniprot_id} not found in DB (Status: {r.status_code})")
                return None

            data = r.json()
            if not data:
                print(f"❌ [AlphaFold Error] No entry found for {uniprot_id}.")
                return None

            # 2. 최신 PDB URL 추출 (첫 번째 항목이 최신)
            pdb_url = data[0]['pdbUrl']
            version = data[0]['latestVersion']
            print(f"   found model v{version} at: {pdb_url}")

            # 3. 파일 다운로드
            save_path = os.path.join(self.download_dir, f"AF-{uniprot_id}.pdb")
            file_data = requests.get(pdb_url).content
            with open(save_path, 'wb') as f:
                f.write(file_data)
            return save_path

        except Exception as e:
            print(f"❌ [Download Error] {e}")
            return None

# --- 2. RMSD 계산 클래스 ---
class StructureAnalyzer:
    def __init__(self):
        self.parser = Bio.PDB.PDBParser(QUIET=True)

    def get_ca_atoms(self, structure):
        return [atom for atom in structure.get_atoms() if atom.name == 'CA']

    def calculate_rmsd(self, file1, file2):
        s1 = self.parser.get_structure("Fixed", file1)
        s2 = self.parser.get_structure("Mobile", file2)

        atoms1 = self.get_ca_atoms(s1)
        atoms2 = self.get_ca_atoms(s2)

        # 길이 맞추기
        min_len = min(len(atoms1), len(atoms2))
        atoms1 = atoms1[:min_len]
        atoms2 = atoms2[:min_len]

        sup = Bio.PDB.Superimposer()
        sup.set_atoms(atoms1, atoms2)
        sup.apply(s2.get_atoms())

        print(f"\n📊 [RMSD 결과]")
        print(f" - 비교 잔기 수: {min_len} 개")
        print(f" - RMSD 값: {sup.rms:.4f} Å")

        io = Bio.PDB.PDBIO()
        io.set_structure(s2)
        aligned_file = file2.replace(".pdb", "_aligned.pdb")
        io.save(aligned_file)

        return sup.rms, aligned_file

# --- 3. 시각화 함수 ---
def show_structures(pdb_file, af_file_aligned):
    view = py3Dmol.view(width=800, height=500)
    with open(pdb_file, 'r') as f:
        view.addModel(f.read(), 'pdb')
    view.setStyle({'model': 0}, {'cartoon': {'color': 'cyan'}})
    view.addLabel("PDB (Exp)", {'position': {'x':0, 'y':0, 'z':0}, 'backgroundColor': 'black', 'fontColor': 'cyan'})

    with open(af_file_aligned, 'r') as f:
        view.addModel(f.read(), 'pdb')
    view.setStyle({'model': 1}, {'cartoon': {'color': 'magenta', 'opacity': 0.8}})
    view.addLabel("AlphaFold (AI)", {'position': {'x':0, 'y':5, 'z':0}, 'backgroundColor': 'black', 'fontColor': 'magenta'})

    view.zoomTo()
    view.show()

In [10]:
# @title 3. 실행하기 (ID 변경됨)

# ---------------- 사용자 입력 설정 ----------------
# [추천] 유비퀴틴 1UBQ와 비교하려면:
target_pdb_id = "1UBQ"       # 실험 구조
target_uniprot_id = "P62987" # AlphaFold (Ubiquitin-60S ribosomal protein L40에서 유비퀴틴 부분 포함)

# [또는 테스트용] 인슐린:
# target_pdb_id = "1TRZ"
# target_uniprot_id = "P01308"
# ------------------------------------------------

fetcher = ProteinFetcher()
path_pdb = fetcher.fetch_from_pdb(target_pdb_id)
path_af = fetcher.fetch_from_alphafold(target_uniprot_id)

if path_pdb and path_af:
    analyzer = StructureAnalyzer()
    rmsd_value, path_af_aligned = analyzer.calculate_rmsd(path_pdb, path_af)
    show_structures(path_pdb, path_af_aligned)
else:
    print("❌ 파일을 찾지 못했습니다. 다른 ID를 시도해보세요.")

📥 [PDB] Fetching 1UBQ...
📥 [AlphaFold] Searching API for P62987...
   found model v6 at: https://alphafold.ebi.ac.uk/files/AF-P62987-F1-model_v6.pdb

📊 [RMSD 결과]
 - 비교 잔기 수: 76 개
 - RMSD 값: 0.7590 Å


3Dmol.js failed to load for some reason. Please check your browser console for error messages.